In [41]:
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

In [50]:
def makeUrl(search, start_date, end_date, start_pg=1, end_pg=143):
    date_part = "&pd=3&ds=" + start_date + "&de=" + end_date
    return ["https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + date_part + "&sort=2&start=" + str((i - 1) * 10 + 1) for i in range(start_pg, end_pg + 1)]

In [51]:
def get_naver_articles(urls):
    articles, titles, dates, press = [], [], [], []
    
    for url in urls:
        html = requests.get(url, headers=headers)
        soup = BeautifulSoup(html.text, "html.parser")

        article_links = [link.attrs['href'] for link in soup.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")]
        article_titles = [title.text for title in soup.select(".news_tit")]
        article_infos = [info.text for info in soup.select(".info")]

        new_urls = []
        idx = 0

        while idx < len(article_links) - 1:
            if 'news.naver.com' in article_links[idx]:
                # Check if the next link also contains 'news.naver.com'
                if idx + 1 < len(article_links) and 'news.naver.com' in article_links[idx + 1]:
                    new_urls.append(article_links[idx + 1])
                    idx += 2
                else:
                    new_urls.append(article_links[idx])
                    idx += 1
            elif 'news.naver.com' in article_links[idx + 1]:
                new_urls.append(article_links[idx + 1])
                idx += 2
            else:
                new_urls.append(article_links[idx])
                idx += 1

        # 마지막 요소 처리
        if idx == len(article_links) - 1:
            new_urls.append(article_links[idx])

        date_pattern = re.compile(r"\d{4}\.\d{2}\.\d{2}")

        current_dates = []
        current_press = []

        for idx, info in enumerate(article_infos):
            match = date_pattern.search(info)
            if match:
                current_dates.append(match.group(0))
                
                if idx > 0:  # 인덱스 0 앞에는 데이터가 없으므로 idx > 0 조건 추가
                    current_press.append(article_infos[idx - 1])

        articles.extend(new_urls)
        titles.extend(article_titles)
        dates.extend(current_dates)
        press.extend(current_press)

        time.sleep(1)
        
    return articles, titles, dates, press

In [52]:
def get_article_contents(article_urls):
    contents = []
    article_time = []  # 이름 변경
    
    for url in tqdm(article_urls):
        if 'news.naver.com' in url:
            news = requests.get(url, headers=headers)
            soup = BeautifulSoup(news.text, "html.parser")

            content = soup.select("#dic_area") or soup.select("#articeBody")
            cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content)).replace("flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", '')
            contents.append(cleaned_content)

            try:
                date = soup.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span").attrs['data-date-time']
            except AttributeError:
                date = re.sub('<[^>]*>', '', str(soup.select_one("#content > div.end_ct > div > div.article_info > span > em")))
            article_time.append(date)  # 이름 변경
        else:
            contents.append('')
            article_time.append('')
    
    return contents, article_time


In [53]:
# def main():
#     search = input("검색할 키워드를 입력해주세요:")
#     start_date = input("\n크롤링할 시작 날짜를 입력해주세요. ex)2022.01.01:")
#     end_date = input("\n크롤링할 종료 날짜를 입력해주세요. ex)2022.12.31:")

#     urls = makeUrl(search, start_date, end_date)
#     article_urls, article_titles, article_dates, article_press = get_naver_articles(urls)
    
#     contents, article_time = get_article_contents(article_urls)  # 이름 변경
    
#     df = pd.DataFrame({'date': article_dates, 'time': article_time, 'title': article_titles, 'content': contents,'press': article_press, 'link': article_urls })  # 이름 변경
#     df.drop_duplicates(keep='first', inplace=True, ignore_index=True)
#     return df

# if __name__ == "__main__":
#     result_df = main()
#     print(result_df)

In [55]:
search = input("검색할 키워드를 입력해주세요:")
start_date = input("\n크롤링할 시작 날짜를 입력해주세요. ex)2022.01.01:")
end_date = input("\n크롤링할 종료 날짜를 입력해주세요. ex)2022.12.31:")

urls = makeUrl(search, start_date, end_date)
article_urls, article_titles, article_dates, article_press = get_naver_articles(urls)

contents, article_time = get_article_contents(article_urls)  # 이름 변경

# 여기서 각 리스트의 길이를 확인합니다.
print("Length of article_urls:", len(article_urls))
print("Length of article_titles:", len(article_titles))
print("Length of article_dates:", len(article_dates))
print("Length of article_press:", len(article_press))
print("Length of contents:", len(contents))
print("Length of article_time:", len(article_time))

# 길이가 동일한지 확인
if len(article_titles) == len(article_dates)== len(article_press) == len(article_urls) == len(contents):
    df = pd.DataFrame({'date': article_dates, 'time': article_time, 'title': article_titles, 'content': contents,'press': article_press, 'link': article_urls }) 
    df.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    print(df)
else:
    print("The lengths of the lists are not equal. Dataframe cannot be created.")



100%|██████████| 1423/1423 [01:08<00:00, 20.68it/s]

Length of article_urls: 1423
Length of article_titles: 1423
Length of article_dates: 1423
Length of article_press: 1423
Length of contents: 1423
Length of article_time: 1423
            date                 time  \
0     2018.08.27                        
1     2018.08.27  2018-08-27 00:01:09   
2     2018.08.27                        
3     2018.08.27                        
4     2018.08.27                        
...          ...                  ...   
1408  2018.08.30                        
1409  2018.08.30                        
1410  2018.08.30  2018-08-30 23:30:58   
1411  2018.08.30                        
1412  2018.08.30                        

                                                  title  \
0                     [2018 한국금융투자포럼] 국내외 블록체인 전문가 한자리에   
1     [가상화폐 뉴스] 08월 27일 00시 00분 비트코인(-0.42%), 질리카(4....   
2     비트코인 -0.29%↓ 이더리움 0.03%↑ 리플 0.27%↑ 대시 2.61%↑ 에...   
3     비트코인 -0.33%↓ 이더리움 -0.13%↓ 리플 -0.27%↓ 퀀텀 -0.21%...   
4     비트코인 -0.45%↓ 이더리움 -0.48%↓ 리플 -0.

In [56]:
df

,date,time,title,content,press,link
0,2018.08.27,,[2018 한국금융투자포럼] 국내외 블록체인 전문가 한자리에,,한국금융신문,http://www.fntimes.com
1,2018.08.27,2018-08-27 00:01:09,"[가상화폐 뉴스] 08월 27일 00시 00분 비트코인(-0.42%), 질리카(4....",\n[한국경제TV 라이온봇 기자]\n\n\n\n[그림 1] 가상화폐 시세 (제공: ...,한국경제TV,https://n.news.naver.com/mnews/article/215/000...
2,2018.08.27,,비트코인 -0.29%↓ 이더리움 0.03%↑ 리플 0.27%↑ 대시 2.61%↑ 에...,,로이슈,http://www.lawissue.co.kr
3,2018.08.27,,비트코인 -0.33%↓ 이더리움 -0.13%↓ 리플 -0.27%↓ 퀀텀 -0.21%...,,로이슈,http://www.lawissue.co.kr
4,2018.08.27,,비트코인 -0.45%↓ 이더리움 -0.48%↓ 리플 -0.27%↓ 라이트코인 -0....,,로이슈,http://www.lawissue.co.kr
...,...,...,...,...,...,...
1408,2018.08.30,,[암호화폐 시황] 비트코인 0.09%↑ · 이더리움 0.81%↑ … 23시 00분 현재,,글로벌경제,http://www.getnews.co.kr
1409,2018.08.30,,"(08월 30일 23시) 거래소별 비트코인, 리플 등 암호화폐 시세 정보",,인사이트코리아,http://www.insightkorea.co.kr/
1410,2018.08.30,2018-08-30 23:30:58,"[가상화폐 뉴스] 08월 30일 23시 30분 비트코인(-2.38%), 스트리머(-...",\n[한국경제TV 라이온봇 기자]\n\n\n\n[그림 1] 가상화폐 시세 (제공: ...,한국경제TV,https://n.news.naver.com/mnews/article/215/000...
1411,2018.08.30,,[암호화폐 시황] 비트코인 0.38%↑ 비트코인캐시 0.17%↑ 비트코인골드 4.7...,,빅데이터뉴스,http://www.thebigdata.co.kr/


In [79]:
# end_date = '2017.10.09'

In [102]:
#데이터 프레임 저장
filename = '{}_{}_{}.csv'.format(search, start_date.replace(".", ""), end_date.replace(".", ""))
df.to_csv(filename, encoding='utf-8-sig', index=False)

In [55]:
article_urls[70:80]

['http://sports.news.naver.com/column/columnList.nhn?expertId=437',
 'https://sports.news.naver.com/news.nhn?oid=260&aid=0000001176',
 'https://n.news.naver.com/mnews/article/008/0003920735?sid=101',
 'https://n.news.naver.com/mnews/article/374/0000137905?sid=101',
 'http://www.digitaltoday.co.kr/',
 'http://www.etoday.co.kr',
 'http://www.etoday.co.kr',
 'http://www.getnews.co.kr',
 'http://www.koreatimes.com',
 'http://www.getnews.co.kr']

In [27]:
article_titles[3990:4000]

['[가상화폐 시황] 비트코인 -0.29%↓·이더리움 0.67%↑...17시 00분 현재',
 '멕시코 중앙은행 총재 “보증없는 비트코인은 화폐 아니다”',
 '비트코인 억만장자 등극한 스웨덴 개발자…비결은?',
 '[가상화폐 시황] 비트코인 0.14%·이더리움 1.06%↑...18시 00분 현재',
 '[클릭 신간] 힐빌리의 노래 외 2권',
 '"자리가 많지 않습니다"…31일 세종대서 은행 공동 취업설명회',
 '에임하이, 한류스타 이름 딴 가상화폐 내놓는다',
 '늘어나는 가상화폐 범죄',
 '[가상화폐 시황] 비트코인 -0.08%·이더리움 1.76%↑...19시 00분 현재',
 '[가상화폐 시황] 비트코인 0.00%·이더리움 -0.39%↓...20시 00분 현재']

In [57]:
df2 = df

In [58]:
df2

,date,time,title,content,press,link
0,2018.08.27,,[2018 한국금융투자포럼] 국내외 블록체인 전문가 한자리에,,한국금융신문,http://www.fntimes.com
1,2018.08.27,2018-08-27 00:01:09,"[가상화폐 뉴스] 08월 27일 00시 00분 비트코인(-0.42%), 질리카(4....",\n[한국경제TV 라이온봇 기자]\n\n\n\n[그림 1] 가상화폐 시세 (제공: ...,한국경제TV,https://n.news.naver.com/mnews/article/215/000...
2,2018.08.27,,비트코인 -0.29%↓ 이더리움 0.03%↑ 리플 0.27%↑ 대시 2.61%↑ 에...,,로이슈,http://www.lawissue.co.kr
3,2018.08.27,,비트코인 -0.33%↓ 이더리움 -0.13%↓ 리플 -0.27%↓ 퀀텀 -0.21%...,,로이슈,http://www.lawissue.co.kr
4,2018.08.27,,비트코인 -0.45%↓ 이더리움 -0.48%↓ 리플 -0.27%↓ 라이트코인 -0....,,로이슈,http://www.lawissue.co.kr
...,...,...,...,...,...,...
1408,2018.08.30,,[암호화폐 시황] 비트코인 0.09%↑ · 이더리움 0.81%↑ … 23시 00분 현재,,글로벌경제,http://www.getnews.co.kr
1409,2018.08.30,,"(08월 30일 23시) 거래소별 비트코인, 리플 등 암호화폐 시세 정보",,인사이트코리아,http://www.insightkorea.co.kr/
1410,2018.08.30,2018-08-30 23:30:58,"[가상화폐 뉴스] 08월 30일 23시 30분 비트코인(-2.38%), 스트리머(-...",\n[한국경제TV 라이온봇 기자]\n\n\n\n[그림 1] 가상화폐 시세 (제공: ...,한국경제TV,https://n.news.naver.com/mnews/article/215/000...
1411,2018.08.30,,[암호화폐 시황] 비트코인 0.38%↑ 비트코인캐시 0.17%↑ 비트코인골드 4.7...,,빅데이터뉴스,http://www.thebigdata.co.kr/


In [97]:
len(df1)
len(df2)
len(df3)
print(len(df1)+len(df2)+len(df3)+len(df4)+len(df5)+len(df6))

15082


In [59]:
result = pd.concat([df1, df2], ignore_index=True)
result.drop_duplicates(keep='first', inplace=True, ignore_index=True)
result

,date,time,title,content,press,link
0,2018.08.17,,비트코인 -0.12%↓ 이더리움 -0.80%↓ 리플 0.30%↑ 대시 -0.67%↓...,,로이슈,http://www.lawissue.co.kr
1,2018.08.17,,비트코인 -0.49%↓ 이더리움 -1.18%↓ 리플 0% 라이트코인 -0.61%↓ ...,,로이슈,http://www.lawissue.co.kr
2,2018.08.17,,비트코인 -0.15%↓ 이더리움 -0.77%↓ 리플 0% 퀀텀 -0.39%↓ 라이트...,,로이슈,http://www.lawissue.co.kr
3,2018.08.17,2018-08-17 00:01:00,"[가상화폐 뉴스] 08월 17일 00시 00분 비트코인(-1.07%), 이더리움 클...",\n[한국경제TV 라이온봇 기자]\n\n\n\n[그림 1] 가상화폐 시세 (제공: ...,한국경제TV,https://n.news.naver.com/mnews/article/215/000...
4,2018.08.17,,[암호화폐 시황] 비트코인 0.51%↓ · 이더리움 1.18%↓ … 00시 00분 현재,,글로벌경제,http://www.getnews.co.kr
...,...,...,...,...,...,...
4998,2018.08.30,,[암호화폐 시황] 비트코인 0.09%↑ · 이더리움 0.81%↑ … 23시 00분 현재,,글로벌경제,http://www.getnews.co.kr
4999,2018.08.30,,"(08월 30일 23시) 거래소별 비트코인, 리플 등 암호화폐 시세 정보",,인사이트코리아,http://www.insightkorea.co.kr/
5000,2018.08.30,2018-08-30 23:30:58,"[가상화폐 뉴스] 08월 30일 23시 30분 비트코인(-2.38%), 스트리머(-...",\n[한국경제TV 라이온봇 기자]\n\n\n\n[그림 1] 가상화폐 시세 (제공: ...,한국경제TV,https://n.news.naver.com/mnews/article/215/000...
5001,2018.08.30,,[암호화폐 시황] 비트코인 0.38%↑ 비트코인캐시 0.17%↑ 비트코인골드 4.7...,,빅데이터뉴스,http://www.thebigdata.co.kr/


In [60]:
result.to_csv('비트코인_20180817_20180830.csv', encoding='utf-8-sig', index=False)